<a href="https://colab.research.google.com/github/shymaagamal/Backpropagation-Algorithm/blob/master/localization_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
data = shuffle(pd.read_csv("/content/Server_main_test.csv"))
data.head(2)

,W1,W2,W3,W4,out
397,84,0,22,66,m3ml
57,0,0,68,0,tor2a_dctra


In [ ]:
Y=data['out']
X=data.drop(['out'], axis=1)

In [ ]:
X_train, X_test, Y_train_label,Y_test_label = train_test_split(X, Y, test_size=0.3, random_state=42)

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

sm = SMOTE()
X_train, Y_train_label = sm.fit_resample(X_train, Y_train_label)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

Normalize the Predictor(Feature Set) for SVM training

In [ ]:
# Transforming non numerical labels into numerical labels
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

# encoding train labels 
encoder.fit(Y_train_label)
Y_train = encoder.transform(Y_train_label)

# encoding test labels 
encoder.fit(Y_test_label)
Y_test = encoder.transform(Y_test_label)

In [ ]:
# #Scaling the Train and Test feature set 
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(quantile_range=(25, 75))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Hyperparameter tuning using grid search and cross validation

In [ ]:
params_grid = [{'kernel': ['rbf'], 'gamma': [0,1e-3, 1e-4,1e-5,10,100],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

### Training SVM model using radial kernel

In [ ]:
svm_model = GridSearchCV(SVC(), params_grid, cv=10)
svm_model.fit(X_train_scaled, Y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0, 0.001, 0.0001, 1e-05, 10, 100],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

### Confusion Matrix  and Accuracy Score 

In [ ]:
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(X_test_scaled)
Y_pred_label = list(encoder.inverse_transform(Y_pred))

Best score for training data: 0.6705521472392638 

Best C: 10 

Best Kernel: rbf 

Best Gamma: 10 



In [ ]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(Y_test_label,Y_pred_label))
print("\n")
print(classification_report(Y_test_label,Y_pred_label))

print("Training set score for SVM: %f" % final_model.score(X_train_scaled , Y_train))
print("Testing  set score for SVM: %f" % final_model.score(X_test_scaled  , Y_test ))

svm_model.score

[[11 26  1  1  4 19  0  0  0  0]
 [14 32  0  1  9  4  0  0  0  0]
 [ 0  0 58  2  0  0  0  0  2  6]
 [ 2  8  0 29  2  0  0  2  1  8]
 [ 2  1  0  3 44  7  0  0  0  0]
 [ 7  9  0  0  9 39  0  0  0  0]
 [ 0  0  0  0  7  0 40  0 14  0]
 [ 0  0  2  0  0  0  0 42  2 11]
 [ 0  0  0  0  2  0  9  1 48  0]
 [ 0  1 10  9  0  0  0 10  0 35]]


                 precision    recall  f1-score   support

   asanser_7mam       0.31      0.18      0.22        62
   bra_bab_7ded       0.42      0.53      0.47        60
           m3ml       0.82      0.85      0.83        68
  mo3deen-tor2a       0.64      0.56      0.60        52
     mo3den_oda       0.57      0.77      0.66        57
        mo3eera       0.57      0.61      0.59        64
     tb3a_dctra       0.82      0.66      0.73        61
tor2a_bra_dctra       0.76      0.74      0.75        57
    tor2a_dctra       0.72      0.80      0.76        60
     tor2a_m3ml       0.58      0.54      0.56        65

       accuracy                       

<bound method BaseSearchCV.score of GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0, 0.001, 0.0001, 1e-05, 10, 100],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)>

# save the model to disk

In [ ]:
filename = 'model68test.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test_scaled  , Y_test )
print(result)

0.6801801801801802


In [ ]:

input_scaled = scaler.transform(input)
output = final_model.predict(input_scaled)
output = list(encoder.inverse_transform(output))